Filon integration
=======

In [1]:
import numpy as np
from scipy.special import sici
import filon
import matplotlib.pyplot as plt
import time
times={}

/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm


In [2]:
def testFunc(x,r,a):
    return np.log(a*x)*np.cos(x*r)

In [3]:
plotting = False
args = np.linspace(1,2.0*np.pi,3000)
if plotting:
    for el in np.arange(10,200,10):
        fig = plt.figure(figsize=(13,10))
        for el2 in [4,5,6,7,8]:
            vals = testFunc(args,el,el2)
            plt.plot(args,vals,label="%s"%el2)
        plt.title("$r = %s$"%el)
        plt.show()


Single evaluation
==========

Exact solution
---------

In [4]:
def exactINDEF(r,a,x):
    return (np.log(a*x)*np.sin(r*x) - sici(r*x)[0])/r

def exact(r,a,low,up):
    return exactINDEF(r,a,up) - exactINDEF(r,a,low)

start = time.time()
resE = exact(np.arange(10,200,10),10,1,2.0*np.pi)
end = time.time() - start
times['exact'] = end

Solution with Filon
---------

In [5]:
def integrand(a,x):
    return np.log(a*x)

def filonInt(r,a,low,up,n=125):
    tempArgs,steps = np.linspace(low,up,n,retstep=True,endpoint=True)
    intArr = integrand(a,tempArgs)
    #print intArr.shape
    res = filon.cos_integral(intArr,steps,r,low)
    return res
start = time.time()
resF = filonInt(np.arange(10,200,10),10,1,2.0*np.pi,125)
end = time.time() - start
times['filon']=end

In [6]:
print "="*30 + " test against the exact solution" + "="*30
for i,el in enumerate(resF):
    print "%1.4e \t%1.4e \t| %1.4e \t%1.4e"%(el,resE[i],resE[i] - el,np.abs((resE[i] - el)/(resE[i] + el)))
print "-"*80
print "="*30 + " time against the exact solution" + "="*30
calls = len(resF)
print "full time (%d calls):  %1.4e | %1.4e"%(calls,times['filon'],times['exact'])
print "time per call:         %1.4e | %1.4e"%(times['filon']/calls,times['exact']/calls)

============================== test against the exact solution==============================
1.3561e-01 	1.3561e-01 	| -1.3122e-07 	4.8383e-07
-1.0584e-01 	-1.0584e-01 	| 7.2951e-08 	3.4464e-07
7.5876e-02 	7.5876e-02 	| 2.2610e-08 	1.4900e-07
-4.2388e-02 	-4.2388e-02 	| -1.3941e-07 	1.6445e-06
1.1763e-02 	1.1763e-02 	| 2.5374e-07 	1.0786e-05
1.2008e-02 	1.2008e-02 	| -3.5321e-07 	1.4707e-05
-2.5556e-02 	-2.5555e-02 	| 1.2166e-07 	2.3803e-06
2.8650e-02 	2.8650e-02 	| 3.2440e-07 	5.6614e-06
-2.2799e-02 	-2.2799e-02 	| 7.6014e-08 	1.6671e-06
1.1590e-02 	1.1590e-02 	| -1.1427e-07 	4.9297e-06
1.0218e-03 	1.0219e-03 	| 8.7547e-08 	4.2839e-05
-1.1187e-02 	-1.1187e-02 	| -4.3010e-08 	1.9224e-06
1.6506e-02 	1.6506e-02 	| 8.5767e-09 	2.5981e-07
-1.6104e-02 	-1.6104e-02 	| -2.1496e-08 	6.6741e-07
1.0950e-02 	1.0950e-02 	| -1.5298e-08 	6.9855e-07
-3.1136e-03 	-3.1135e-03 	| 7.0100e-08 	1.1257e-05
-4.7222e-03 	-4.7223e-03 	| -4.8892e-08 	5.1768e-06
1.0272e-02 	1.0272e-02 	| 2.3933e-08 	1.1649e-06
-

Array evaluation
===========

Exact solution
---------

In [7]:
rArr = np.arange(10,200,1)
aArr = np.arange(101) + np.ones(101)
rGrid = rArr[:,np.newaxis]
aGrid = aArr[np.newaxis,:]
start = time.time()
resEall = exact(rGrid,aGrid,1,2.0*np.pi)
end = time.time() - start
times['exactAll'] = end

Solution with Filon
---------

In [8]:
def filonIntAll(r,a,low,up,n=1001): 
    aGrid = a[np.newaxis,:]
    args,steps = np.linspace(low,up,n,retstep=True,endpoint=True)
    argsGrid = args[:,np.newaxis]
    intArr = integrand(aGrid,argsGrid)
    print intArr.shape
    res = filon.cos_integral(intArr,steps,r,low,axis=0)
    #print a.shape
    return res

rArr = np.arange(10,200,1)
aArr = np.arange(101) + np.ones(101)
print rArr
start = time.time()
resFall = filonIntAll(rArr,aArr,1,2.0*np.pi)
end = time.time() - start
times['filonAll'] = end

[ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199]
(1001, 101)


Compare Filon with exact
-------

In [9]:
ind = 0
for ir,rel in enumerate(rArr):
    for ia,ael in enumerate(aArr):
        print "%d:\t %1.4e\t%1.4e\t| %1.4e \t%1.4e \t| %1.4e \t%1.4e"%(ind,rel,ael,resFall[ir][ia],resEall[ir][ia],resFall[ir][ia]-resEall[ir][ia],np.abs((resFall[ir][ia]-resEall[ir][ia])/(resFall[ir][ia]+resEall[ir][ia])))
        ind+=1
calls = len(rArr) * len(aArr)
print "full time (%d calls):  \t%1.4e | %1.4e"%(calls,times['filonAll'],times['exactAll'])
print "time per call:         \t\t%1.4e | %1.4e"%(times['filonAll']/calls,times['exactAll']/calls)

0:	 1.0000e+01	1.0000e+00	| 1.0346e-02 	1.0346e-02 	| 3.0728e-11 	1.4851e-09
1:	 1.0000e+01	2.0000e+00	| 4.8055e-02 	4.8055e-02 	| 3.0725e-11 	3.1969e-10
2:	 1.0000e+01	3.0000e+00	| 7.0113e-02 	7.0113e-02 	| 3.0723e-11 	2.1910e-10
3:	 1.0000e+01	4.0000e+00	| 8.5763e-02 	8.5763e-02 	| 3.0722e-11 	1.7911e-10
4:	 1.0000e+01	5.0000e+00	| 9.7903e-02 	9.7903e-02 	| 3.0721e-11 	1.5689e-10
5:	 1.0000e+01	6.0000e+00	| 1.0782e-01 	1.0782e-01 	| 3.0720e-11 	1.4246e-10
6:	 1.0000e+01	7.0000e+00	| 1.1621e-01 	1.1621e-01 	| 3.0719e-11 	1.3217e-10
7:	 1.0000e+01	8.0000e+00	| 1.2347e-01 	1.2347e-01 	| 3.0718e-11 	1.2439e-10
8:	 1.0000e+01	9.0000e+00	| 1.2988e-01 	1.2988e-01 	| 3.0718e-11 	1.1826e-10
9:	 1.0000e+01	1.0000e+01	| 1.3561e-01 	1.3561e-01 	| 3.0717e-11 	1.1326e-10
10:	 1.0000e+01	1.1000e+01	| 1.4080e-01 	1.4080e-01 	| 3.0717e-11 	1.0908e-10
11:	 1.0000e+01	1.2000e+01	| 1.4553e-01 	1.4553e-01 	| 3.0716e-11 	1.0553e-10
12:	 1.0000e+01	1.3000e+01	| 1.4988e-01 	1.4988e-01 	| 3.0716e-11 	1.0247e

1072:	 2.0000e+01	6.3000e+01	| -1.8985e-01 	-1.8985e-01 	| -1.7086e-11 	4.4998e-11
1073:	 2.0000e+01	6.4000e+01	| -1.9057e-01 	-1.9057e-01 	| -1.7085e-11 	4.4827e-11
1074:	 2.0000e+01	6.5000e+01	| -1.9128e-01 	-1.9128e-01 	| -1.7086e-11 	4.4662e-11
1075:	 2.0000e+01	6.6000e+01	| -1.9198e-01 	-1.9198e-01 	| -1.7086e-11 	4.4500e-11
1076:	 2.0000e+01	6.7000e+01	| -1.9266e-01 	-1.9266e-01 	| -1.7085e-11 	4.4340e-11
1077:	 2.0000e+01	6.8000e+01	| -1.9334e-01 	-1.9334e-01 	| -1.7085e-11 	4.4185e-11
1078:	 2.0000e+01	6.9000e+01	| -1.9401e-01 	-1.9401e-01 	| -1.7085e-11 	4.4033e-11
1079:	 2.0000e+01	7.0000e+01	| -1.9466e-01 	-1.9466e-01 	| -1.7085e-11 	4.3884e-11
1080:	 2.0000e+01	7.1000e+01	| -1.9531e-01 	-1.9531e-01 	| -1.7085e-11 	4.3739e-11
1081:	 2.0000e+01	7.2000e+01	| -1.9595e-01 	-1.9595e-01 	| -1.7086e-11 	4.3597e-11
1082:	 2.0000e+01	7.3000e+01	| -1.9658e-01 	-1.9658e-01 	| -1.7085e-11 	4.3457e-11
1083:	 2.0000e+01	7.4000e+01	| -1.9720e-01 	-1.9720e-01 	| -1.7086e-11 	4.3321e-11
1084

1972:	 2.9000e+01	5.4000e+01	| 9.2387e-02 	9.2387e-02 	| 2.7568e-11 	1.4920e-10
1973:	 2.9000e+01	5.5000e+01	| 9.2807e-02 	9.2807e-02 	| 2.7568e-11 	1.4852e-10
1974:	 2.9000e+01	5.6000e+01	| 9.3220e-02 	9.3220e-02 	| 2.7568e-11 	1.4786e-10
1975:	 2.9000e+01	5.7000e+01	| 9.3625e-02 	9.3625e-02 	| 2.7568e-11 	1.4723e-10
1976:	 2.9000e+01	5.8000e+01	| 9.4023e-02 	9.4023e-02 	| 2.7568e-11 	1.4660e-10
1977:	 2.9000e+01	5.9000e+01	| 9.4414e-02 	9.4414e-02 	| 2.7568e-11 	1.4599e-10
1978:	 2.9000e+01	6.0000e+01	| 9.4798e-02 	9.4798e-02 	| 2.7568e-11 	1.4540e-10
1979:	 2.9000e+01	6.1000e+01	| 9.5177e-02 	9.5177e-02 	| 2.7568e-11 	1.4482e-10
1980:	 2.9000e+01	6.2000e+01	| 9.5549e-02 	9.5549e-02 	| 2.7568e-11 	1.4426e-10
1981:	 2.9000e+01	6.3000e+01	| 9.5915e-02 	9.5915e-02 	| 2.7568e-11 	1.4371e-10
1982:	 2.9000e+01	6.4000e+01	| 9.6275e-02 	9.6275e-02 	| 2.7568e-11 	1.4317e-10
1983:	 2.9000e+01	6.5000e+01	| 9.6630e-02 	9.6630e-02 	| 2.7568e-11 	1.4265e-10
1984:	 2.9000e+01	6.6000e+01	| 9.6979e-0

2886:	 3.8000e+01	5.9000e+01	| -3.2357e-02 	-3.2357e-02 	| -3.3449e-11 	5.1688e-10
2887:	 3.8000e+01	6.0000e+01	| -3.2488e-02 	-3.2488e-02 	| -3.3449e-11 	5.1479e-10
2888:	 3.8000e+01	6.1000e+01	| -3.2617e-02 	-3.2617e-02 	| -3.3449e-11 	5.1276e-10
2889:	 3.8000e+01	6.2000e+01	| -3.2744e-02 	-3.2744e-02 	| -3.3449e-11 	5.1077e-10
2890:	 3.8000e+01	6.3000e+01	| -3.2869e-02 	-3.2869e-02 	| -3.3449e-11 	5.0883e-10
2891:	 3.8000e+01	6.4000e+01	| -3.2992e-02 	-3.2992e-02 	| -3.3450e-11 	5.0694e-10
2892:	 3.8000e+01	6.5000e+01	| -3.3112e-02 	-3.3112e-02 	| -3.3449e-11 	5.0509e-10
2893:	 3.8000e+01	6.6000e+01	| -3.3232e-02 	-3.3232e-02 	| -3.3449e-11 	5.0328e-10
2894:	 3.8000e+01	6.7000e+01	| -3.3349e-02 	-3.3349e-02 	| -3.3449e-11 	5.0151e-10
2895:	 3.8000e+01	6.8000e+01	| -3.3464e-02 	-3.3464e-02 	| -3.3449e-11 	4.9978e-10
2896:	 3.8000e+01	6.9000e+01	| -3.3578e-02 	-3.3578e-02 	| -3.3449e-11 	4.9808e-10
2897:	 3.8000e+01	7.0000e+01	| -3.3690e-02 	-3.3690e-02 	| -3.3450e-11 	4.9643e-10
2898

3578:	 4.5000e+01	4.4000e+01	| -7.1745e-02 	-7.1745e-02 	| -1.9490e-11 	1.3583e-10
3579:	 4.5000e+01	4.5000e+01	| -7.2170e-02 	-7.2170e-02 	| -1.9490e-11 	1.3503e-10
3580:	 4.5000e+01	4.6000e+01	| -7.2586e-02 	-7.2586e-02 	| -1.9491e-11 	1.3426e-10
3581:	 4.5000e+01	4.7000e+01	| -7.2992e-02 	-7.2992e-02 	| -1.9491e-11 	1.3351e-10
3582:	 4.5000e+01	4.8000e+01	| -7.3390e-02 	-7.3390e-02 	| -1.9490e-11 	1.3278e-10
3583:	 4.5000e+01	4.9000e+01	| -7.3780e-02 	-7.3780e-02 	| -1.9490e-11 	1.3208e-10
3584:	 4.5000e+01	5.0000e+01	| -7.4162e-02 	-7.4162e-02 	| -1.9490e-11 	1.3140e-10
3585:	 4.5000e+01	5.1000e+01	| -7.4537e-02 	-7.4537e-02 	| -1.9490e-11 	1.3074e-10
3586:	 4.5000e+01	5.2000e+01	| -7.4904e-02 	-7.4904e-02 	| -1.9490e-11 	1.3010e-10
3587:	 4.5000e+01	5.3000e+01	| -7.5264e-02 	-7.5264e-02 	| -1.9490e-11 	1.2948e-10
3588:	 4.5000e+01	5.4000e+01	| -7.5617e-02 	-7.5617e-02 	| -1.9490e-11 	1.2887e-10
3589:	 4.5000e+01	5.5000e+01	| -7.5964e-02 	-7.5964e-02 	| -1.9490e-11 	1.2828e-10
3590

...

15502:	 1.6300e+02	5.0000e+01	| 8.4888e-03 	8.4888e-03 	| -3.4551e-11 	2.0351e-09
15503:	 1.6300e+02	5.1000e+01	| 8.5319e-03 	8.5319e-03 	| -3.4551e-11 	2.0248e-09
15504:	 1.6300e+02	5.2000e+01	| 8.5742e-03 	8.5742e-03 	| -3.4551e-11 	2.0148e-09
15505:	 1.6300e+02	5.3000e+01	| 8.6157e-03 	8.6157e-03 	| -3.4551e-11 	2.0051e-09
15506:	 1.6300e+02	5.4000e+01	| 8.6564e-03 	8.6564e-03 	| -3.4551e-11 	1.9957e-09
15507:	 1.6300e+02	5.5000e+01	| 8.6963e-03 	8.6963e-03 	| -3.4551e-11 	1.9865e-09
15508:	 1.6300e+02	5.6000e+01	| 8.7355e-03 	8.7355e-03 	| -3.4551e-11 	1.9776e-09
15509:	 1.6300e+02	5.7000e+01	| 8.7741e-03 	8.7741e-03 	| -3.4551e-11 	1.9689e-09
15510:	 1.6300e+02	5.8000e+01	| 8.8119e-03 	8.8119e-03 	| -3.4551e-11 	1.9605e-09
15511:	 1.6300e+02	5.9000e+01	| 8.8492e-03 	8.8492e-03 	| -3.4551e-11 	1.9522e-09
15512:	 1.6300e+02	6.0000e+01	| 8.8858e-03 	8.8858e-03 	| -3.4551e-11 	1.9442e-09
15513:	 1.6300e+02	6.1000e+01	| 8.9218e-03 	8.9218e-03 	| -3.4551e-11 	1.9363e-09
15514:	 1.6300e+

16178:	 1.7000e+02	1.9000e+01	| -6.0311e-03 	-6.0311e-03 	| -3.5137e-11 	2.9130e-09
16179:	 1.7000e+02	2.0000e+01	| -6.1357e-03 	-6.1357e-03 	| -3.5136e-11 	2.8633e-09
16180:	 1.7000e+02	2.1000e+01	| -6.2351e-03 	-6.2351e-03 	| -3.5136e-11 	2.8176e-09
16181:	 1.7000e+02	2.2000e+01	| -6.3300e-03 	-6.3300e-03 	| -3.5136e-11 	2.7754e-09
16182:	 1.7000e+02	2.3000e+01	| -6.4206e-03 	-6.4206e-03 	| -3.5136e-11 	2.7362e-09
16183:	 1.7000e+02	2.4000e+01	| -6.5074e-03 	-6.5074e-03 	| -3.5136e-11 	2.6997e-09
16184:	 1.7000e+02	2.5000e+01	| -6.5907e-03 	-6.5907e-03 	| -3.5136e-11 	2.6656e-09
16185:	 1.7000e+02	2.6000e+01	| -6.6706e-03 	-6.6706e-03 	| -3.5136e-11 	2.6336e-09
16186:	 1.7000e+02	2.7000e+01	| -6.7476e-03 	-6.7476e-03 	| -3.5136e-11 	2.6036e-09
16187:	 1.7000e+02	2.8000e+01	| -6.8218e-03 	-6.8218e-03 	| -3.5136e-11 	2.5753e-09
16188:	 1.7000e+02	2.9000e+01	| -6.8933e-03 	-6.8933e-03 	| -3.5136e-11 	2.5485e-09
16189:	 1.7000e+02	3.0000e+01	| -6.9624e-03 	-6.9624e-03 	| -3.5136e-11 	2.5

16821:	 1.7600e+02	5.6000e+01	| -1.6453e-03 	-1.6453e-03 	| -3.7506e-11 	1.1398e-08
16822:	 1.7600e+02	5.7000e+01	| -1.6524e-03 	-1.6524e-03 	| -3.7506e-11 	1.1349e-08
16823:	 1.7600e+02	5.8000e+01	| -1.6594e-03 	-1.6594e-03 	| -3.7506e-11 	1.1301e-08
16824:	 1.7600e+02	5.9000e+01	| -1.6663e-03 	-1.6663e-03 	| -3.7506e-11 	1.1254e-08
16825:	 1.7600e+02	6.0000e+01	| -1.6730e-03 	-1.6730e-03 	| -3.7506e-11 	1.1209e-08
16826:	 1.7600e+02	6.1000e+01	| -1.6797e-03 	-1.6797e-03 	| -3.7505e-11 	1.1165e-08
16827:	 1.7600e+02	6.2000e+01	| -1.6862e-03 	-1.6862e-03 	| -3.7505e-11 	1.1121e-08
16828:	 1.7600e+02	6.3000e+01	| -1.6926e-03 	-1.6926e-03 	| -3.7506e-11 	1.1079e-08
16829:	 1.7600e+02	6.4000e+01	| -1.6990e-03 	-1.6990e-03 	| -3.7506e-11 	1.1038e-08
16830:	 1.7600e+02	6.5000e+01	| -1.7052e-03 	-1.7052e-03 	| -3.7506e-11 	1.0997e-08
16831:	 1.7600e+02	6.6000e+01	| -1.7113e-03 	-1.7113e-03 	| -3.7506e-11 	1.0958e-08
16832:	 1.7600e+02	6.7000e+01	| -1.7174e-03 	-1.7174e-03 	| -3.7506e-11 	1.0

17625:	 1.8400e+02	5.2000e+01	| -2.0960e-02 	-2.0960e-02 	| 8.0836e-12 	1.9283e-10
17626:	 1.8400e+02	5.3000e+01	| -2.1062e-02 	-2.1062e-02 	| 8.0836e-12 	1.9191e-10
17627:	 1.8400e+02	5.4000e+01	| -2.1161e-02 	-2.1161e-02 	| 8.0836e-12 	1.9100e-10
17628:	 1.8400e+02	5.5000e+01	| -2.1258e-02 	-2.1258e-02 	| 8.0838e-12 	1.9013e-10
17629:	 1.8400e+02	5.6000e+01	| -2.1354e-02 	-2.1354e-02 	| 8.0838e-12 	1.8928e-10
17630:	 1.8400e+02	5.7000e+01	| -2.1448e-02 	-2.1448e-02 	| 8.0838e-12 	1.8845e-10
17631:	 1.8400e+02	5.8000e+01	| -2.1540e-02 	-2.1540e-02 	| 8.0839e-12 	1.8765e-10
17632:	 1.8400e+02	5.9000e+01	| -2.1631e-02 	-2.1631e-02 	| 8.0838e-12 	1.8686e-10
17633:	 1.8400e+02	6.0000e+01	| -2.1720e-02 	-2.1720e-02 	| 8.0840e-12 	1.8610e-10
17634:	 1.8400e+02	6.1000e+01	| -2.1808e-02 	-2.1808e-02 	| 8.0840e-12 	1.8535e-10
17635:	 1.8400e+02	6.2000e+01	| -2.1894e-02 	-2.1894e-02 	| 8.0840e-12 	1.8462e-10
17636:	 1.8400e+02	6.3000e+01	| -2.1979e-02 	-2.1979e-02 	| 8.0840e-12 	1.8390e-10
1763

18581:	 1.9300e+02	9.9000e+01	| 2.3306e-02 	2.3306e-02 	| 8.3303e-12 	1.7872e-10
18582:	 1.9300e+02	1.0000e+02	| 2.3357e-02 	2.3357e-02 	| 8.3303e-12 	1.7833e-10
18583:	 1.9300e+02	1.0100e+02	| 2.3407e-02 	2.3407e-02 	| 8.3303e-12 	1.7794e-10
18584:	 1.9400e+02	1.0000e+00	| -1.4587e-05 	-1.4587e-05 	| -2.7073e-11 	9.2801e-07
18585:	 1.9400e+02	2.0000e+00	| 2.4950e-03 	2.4950e-03 	| -2.7075e-11 	5.4259e-09
18586:	 1.9400e+02	3.0000e+00	| 3.9630e-03 	3.9630e-03 	| -2.7076e-11 	3.4161e-09
18587:	 1.9400e+02	4.0000e+00	| 5.0046e-03 	5.0046e-03 	| -2.7076e-11 	2.7052e-09
18588:	 1.9400e+02	5.0000e+00	| 5.8125e-03 	5.8125e-03 	| -2.7077e-11 	2.3292e-09
18589:	 1.9400e+02	6.0000e+00	| 6.4726e-03 	6.4726e-03 	| -2.7078e-11 	2.0917e-09
18590:	 1.9400e+02	7.0000e+00	| 7.0307e-03 	7.0307e-03 	| -2.7078e-11 	1.9257e-09
18591:	 1.9400e+02	8.0000e+00	| 7.5141e-03 	7.5141e-03 	| -2.7078e-11 	1.8018e-09
18592:	 1.9400e+02	9.0000e+00	| 7.9406e-03 	7.9406e-03 	| -2.7078e-11 	1.7051e-09
18593:	 1.9400e+0

In [10]:
relErr = np.abs((resFall-resEall)/(resFall+resEall))
i,j = np.unravel_index(relErr.argmax(), relErr.shape)
print "max. relErr: (%1.4e,%1.4e) %1.4e"%(rArr[i],aArr[j],relErr[i,j])

max. relErr: (7.4000e+01,1.0000e+00) 1.6812e-05
